<a href="https://colab.research.google.com/github/HansAzharr/ML-Material-Screening/blob/main/03_Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib # For saving and loading models and scaler

from sklearn.multioutput import MultiOutputRegressor # import for multi-output handling
from sklearn.ensemble import RandomForestRegressor


## Load file

In [ ]:
data_folder_path = '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/'
engineered_features_filename = os.path.join(data_folder_path, "battery_materials_engineered_features_v2.csv")

# Load the engineered data
df_engineered = pd.read_csv(engineered_features_filename)
print(f"Successfully loaded engineered features DataFrame from '{engineered_features_filename}'.")
print(f"DataFrame shape: {df_engineered.shape}")

Successfully loaded engineered features DataFrame from '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/battery_materials_engineered_features_v2.csv'.
DataFrame shape: (3445, 53)


In [ ]:
print(df_engineered.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3445 entries, 0 to 3444
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   material_id                  3445 non-null   object 
 1   energy_above_hull            3445 non-null   float64
 2   density                      3445 non-null   float64
 3   formula_pretty               3445 non-null   object 
 4   nelements                    3445 non-null   float64
 5   volume                       3445 non-null   float64
 6   band_gap                     3445 non-null   float64
 7   total_magnetization          3445 non-null   float64
 8   battery_type                 3445 non-null   object 
 9   average_voltage              3445 non-null   float64
 10  theoretical_capacity_grav    3445 non-null   float64
 11  theoretical_capacity_vol     3445 non-null   float64
 12  energy_grav                  3445 non-null   float64
 13  energy_vol        

## Define Features (X) and Targets (Y)

In [ ]:
Y_cols = ['theoretical_capacity_grav', 'average_voltage', 'energy_above_hull']

excluded_cols = ['material_id', 'formula_pretty', 'battery_type', 'theoretical_capacity_vol', 'stability_charge'] + Y_cols

X_cols = [col for col in df_engineered.columns if col not in excluded_cols]

X = df_engineered[X_cols]
Y = df_engineered[Y_cols]

print(f"\nDefined X (features) with {X.shape[1]} columns and Y (targets) with {Y.shape[1]} columns.")
print("X (features) columns:", X.columns.tolist())
print("Y (targets) columns:", Y.columns.tolist())



Defined X (features) with 45 columns and Y (targets) with 3 columns.
X (features) columns: ['density', 'nelements', 'volume', 'band_gap', 'total_magnetization', 'energy_grav', 'energy_vol', 'num_steps', 'max_voltage_step', 'stability_discharge', 'e_fermi', 'nsites', 'num_unique_elements', 'avg_atomic_number', 'min_atomic_number', 'max_atomic_number', 'avg_atomic_mass', 'min_atomic_mass', 'max_atomic_mass', 'avg_electronegativity', 'min_electronegativity', 'max_electronegativity', 'avg_covalent_radius', 'min_covalent_radius', 'max_covalent_radius', 'std_electronegativity', 'ionic_radius_ion', 'atomic_weight_ion', 'charge_ion', 'atomic_number_ion', 'electronegativity_ion', 'working_ion_Li', 'working_ion_Mg', 'working_ion_Na', 'crystal_system_Cubic', 'crystal_system_Hexagonal', 'crystal_system_Monoclinic', 'crystal_system_Orthorhombic', 'crystal_system_Tetragonal', 'crystal_system_Triclinic', 'crystal_system_Trigonal', 'is_metal_False', 'is_metal_True', 'theoretical_False', 'theoretical_

## Train-Test Split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=9)
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

X_train shape: (2756, 45), Y_train shape: (2756, 3)
X_test shape: (689, 45), Y_test shape: (689, 3)


## Feature Scaling (StandardScaler)

In [ ]:
# Select columns that are float64 or int64 dtypes.
numerical_cols_for_scaling = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Further filter out columns that are actually binary (0 or 1) from this list
binary_cols = [col for col in numerical_cols_for_scaling
               if X_train[col].nunique() == 2 and X_train[col].min() == 0 and X_train[col].max() == 1]

# The actual columns to scale are numerical, minus the identified binary columns
numerical_cols_to_scale = [col for col in numerical_cols_for_scaling if col not in binary_cols]

print(f"\nIdentified {len(numerical_cols_to_scale)} numerical columns for scaling:")
print(numerical_cols_to_scale)


Identified 31 numerical columns for scaling:
['density', 'nelements', 'volume', 'band_gap', 'total_magnetization', 'energy_grav', 'energy_vol', 'num_steps', 'max_voltage_step', 'stability_discharge', 'e_fermi', 'nsites', 'num_unique_elements', 'avg_atomic_number', 'min_atomic_number', 'max_atomic_number', 'avg_atomic_mass', 'min_atomic_mass', 'max_atomic_mass', 'avg_electronegativity', 'min_electronegativity', 'max_electronegativity', 'avg_covalent_radius', 'min_covalent_radius', 'max_covalent_radius', 'std_electronegativity', 'ionic_radius_ion', 'atomic_weight_ion', 'charge_ion', 'atomic_number_ion', 'electronegativity_ion']


In [ ]:
# Initialize the StandardScaler
scaler_X = StandardScaler()

# Fit the scaler on the training data ONLY for the identified numerical columns
X_train_scaled = X_train.copy() # Create a copy to store scaled values
X_test_scaled = X_test.copy()   # Create a copy to store scaled values

if numerical_cols_to_scale: # Only scale if there are columns to scale
    X_train_scaled[numerical_cols_to_scale] = scaler_X.fit_transform(X_train[numerical_cols_to_scale])
    # Transform testing data using the scaler fitted on training data
    X_test_scaled[numerical_cols_to_scale] = scaler_X.transform(X_test[numerical_cols_to_scale])
    print("Numerical features scaled successfully.")
else:
    print("No continuous numerical features found for scaling.")
    # If no numerical features, ensure scaled variables are just copies of originals
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

Numerical features scaled successfully.


In [ ]:
# Display head of scaled training features (check a few numerical ones)

# Dynamically select a few numerical features and some encoded ones for display
display_cols = []
if numerical_cols_to_scale:
    display_cols.extend(numerical_cols_to_scale[:3]) # First 3 numerical
# Add a few one-hot encoded columns
encoded_cols_example = [col for col in X_train.columns if 'crystal_system_' in col or 'working_ion_' in col or 'is_metal_' in col]
display_cols.extend(encoded_cols_example[:3]) # First 3 example encoded

# Ensure display_cols are unique and exist in X_train_scaled
display_cols = list(set(col for col in display_cols if col in X_train_scaled.columns))
if display_cols:
    print(X_train_scaled[display_cols].head().to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("No suitable columns to display example scaled data.")

| working_ion_Mg   | working_ion_Li   | volume    | working_ion_Na   | nelements   | density   |
|:-----------------|:-----------------|:----------|:-----------------|:------------|:----------|
| 0                | 1                | -0.404032 | 0                | 0.665793    | -0.169257 |
| 1                | 0                | -1.19194  | 0                | -0.656199   | 0.497746  |
| 0                | 1                | 1.85852   | 0                | -0.656199   | -0.193005 |
| 1                | 0                | -0.894973 | 0                | 0.665793    | 3.36416   |
| 0                | 1                | -0.646339 | 0                | -1.97819    | -0.235335 |


## Adding Interaction features

In [ ]:
# Helper function to add interaction features to both train and test sets
def add_interaction_features(df, feature1, feature2, new_feature_name):
    df[new_feature_name] = df[feature1] * df[feature2]
    return df

# --- Selected Interactions with theoretical_False (Experimental) ---
X_train_scaled = add_interaction_features(X_train_scaled, 'theoretical_False', 'band_gap', 'exp_band_gap')
X_test_scaled = add_interaction_features(X_test_scaled, 'theoretical_False', 'band_gap', 'exp_band_gap')

X_train_scaled = add_interaction_features(X_train_scaled, 'theoretical_False', 'energy_vol', 'exp_energy_vol')
X_test_scaled = add_interaction_features(X_test_scaled, 'theoretical_False', 'energy_vol', 'exp_energy_vol')

# --- Selected Interactions with theoretical_True (Theoretical) ---
X_train_scaled = add_interaction_features(X_train_scaled, 'theoretical_True', 'band_gap', 'theo_band_gap')
X_test_scaled = add_interaction_features(X_test_scaled, 'theoretical_True', 'band_gap', 'theo_band_gap')

X_train_scaled = add_interaction_features(X_train_scaled, 'theoretical_True', 'energy_vol', 'theo_energy_vol')
X_test_scaled = add_interaction_features(X_test_scaled, 'theoretical_True', 'energy_vol', 'theo_energy_vol')

print("Selected 4 new features added to X_train_scaled and X_test_scaled.")
print(f"New total number of features in X_train_scaled: {X_train_scaled.shape[1]}")

Selected 4 new features added to X_train_scaled and X_test_scaled.
New total number of features in X_train_scaled: 49


In [ ]:
X_train_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2756 entries, 2553 to 382
Data columns (total 49 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   density                      2756 non-null   float64
 1   nelements                    2756 non-null   float64
 2   volume                       2756 non-null   float64
 3   band_gap                     2756 non-null   float64
 4   total_magnetization          2756 non-null   float64
 5   energy_grav                  2756 non-null   float64
 6   energy_vol                   2756 non-null   float64
 7   num_steps                    2756 non-null   float64
 8   max_voltage_step             2756 non-null   float64
 9   stability_discharge          2756 non-null   float64
 10  e_fermi                      2756 non-null   float64
 11  nsites                       2756 non-null   float64
 12  num_unique_elements          2756 non-null   float64
 13  avg_atomic_number    

# XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
# Define the base XGBoost model

xgb_base_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=300,  # A good starting point for number of boosting rounds
    learning_rate=0.1, # Step size shrinkage
    max_depth=6,       # Maximum depth of a tree
    subsample=0.8,     # Subsample ratio of the training instance
    colsample_bytree=0.8, # Subsample ratio of columns when constructing each tree
    random_state=9,
    n_jobs=-1
)

In [ ]:
# Wrap the XGBoost model in MultiOutputRegressor
multioutput_xgb = MultiOutputRegressor(xgb_base_model)

# Prepare lists to store scores for each target across all folds
xgb_target_r2_scores = {col: [] for col in Y_train.columns}
xgb_target_mae_scores = {col: [] for col in Y_train.columns}
xgb_target_rmse_scores = {col: [] for col in Y_train.columns}

In [ ]:
# Perform cross-validation with the XGBoost model
kfold = KFold(n_splits=5, shuffle=True, random_state=9)

for fold_idx, (train_index, test_index) in enumerate(kfold.split(X_train_scaled, Y_train)):
    X_train_fold, X_test_fold = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]

    # Fit a new instance of the MultiOutput XGBoost model for each fold
    # This automatically trains a separate XGBoost model for each target
    fold_multioutput_xgb = MultiOutputRegressor(xgb_base_model)
    fold_multioutput_xgb.fit(X_train_fold, Y_train_fold)

    Y_pred_fold = fold_multioutput_xgb.predict(X_test_fold)
    # Convert predictions back to DataFrame with column names for clarity
    Y_pred_fold_df = pd.DataFrame(Y_pred_fold, columns=Y_test_fold.columns, index=Y_test_fold.index)

    for i, target_col in enumerate(Y_test_fold.columns):
        r2 = r2_score(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        mae = mean_absolute_error(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        rmse = np.sqrt(mean_squared_error(Y_test_fold[target_col], Y_pred_fold_df[target_col]))

        xgb_target_r2_scores[target_col].append(r2)
        xgb_target_mae_scores[target_col].append(mae)
        xgb_target_rmse_scores[target_col].append(rmse)

In [ ]:
# Print aggregated cross-validation results for XGBoost
print("\n--- XGBoost Cross-Validation Results (Mean across 5 folds, Per Target) ---")
for target_col in Y_train.columns:
    print(f"\nMetrics for Target: {target_col}")
    print(f"  Mean R-squared (R²): {np.mean(xgb_target_r2_scores[target_col]):.4f} (Std Dev: {np.std(xgb_target_r2_scores[target_col]):.4f})")
    print(f"  Mean MAE: {np.mean(xgb_target_mae_scores[target_col]):.4f} (Std Dev: {np.std(xgb_target_mae_scores[target_col]):.4f})")
    print(f"  Mean RMSE: {np.mean(xgb_target_rmse_scores[target_col]):.4f} (Std Dev: {np.std(xgb_target_rmse_scores[target_col]):.4f})")


--- XGBoost Cross-Validation Results (Mean across 5 folds, Per Target) ---

Metrics for Target: theoretical_capacity_grav
  Mean R-squared (R²): 0.8422 (Std Dev: 0.0444)
  Mean MAE: 20.7343 (Std Dev: 0.9978)
  Mean RMSE: 56.5900 (Std Dev: 11.3620)

Metrics for Target: average_voltage
  Mean R-squared (R²): 0.8030 (Std Dev: 0.1680)
  Mean MAE: 0.4322 (Std Dev: 0.1859)
  Mean RMSE: 2.8554 (Std Dev: 3.7643)

Metrics for Target: energy_above_hull
  Mean R-squared (R²): 0.4669 (Std Dev: 0.0264)
  Mean MAE: 0.0244 (Std Dev: 0.0007)
  Mean RMSE: 0.0331 (Std Dev: 0.0008)


In [ ]:
# Train on the full training set and evaluate on test set for a final view

print("\n--- XGBoost Model Evaluation on Test Set ---")
multioutput_xgb_final = MultiOutputRegressor(xgb_base_model)
multioutput_xgb_final.fit(X_train_scaled, Y_train) # Train on full training data

Y_pred_xgb_test = multioutput_xgb_final.predict(X_test_scaled)
Y_pred_xgb_test_df = pd.DataFrame(Y_pred_xgb_test, columns=Y_test.columns, index=Y_test.index)

for i, target_col in enumerate(Y_test.columns):
    r2 = r2_score(Y_test[target_col], Y_pred_xgb_test_df[target_col])
    mae = mean_absolute_error(Y_test[target_col], Y_pred_xgb_test_df[target_col])
    rmse = np.sqrt(mean_squared_error(Y_test[target_col], Y_pred_xgb_test_df[target_col]))

    print(f"\nMetrics for Target: {target_col} (Test Set)")
    print(f"  R-squared (R²): {r2:.4f}")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")


--- XGBoost Model Evaluation on Test Set ---

Metrics for Target: theoretical_capacity_grav (Test Set)
  R-squared (R²): 0.7347
  Mean Absolute Error (MAE): 17.4324
  Root Mean Squared Error (RMSE): 36.7951

Metrics for Target: average_voltage (Test Set)
  R-squared (R²): 0.8565
  Mean Absolute Error (MAE): 0.3703
  Root Mean Squared Error (RMSE): 1.5015

Metrics for Target: energy_above_hull (Test Set)
  R-squared (R²): 0.5203
  Mean Absolute Error (MAE): 0.0232
  Root Mean Squared Error (RMSE): 0.0319


## Testing on dataset

In [ ]:
import random

In [ ]:
# Select a random index from the test set
random_index = random.randint(0, len(X_test_scaled) - 1)

# Get the actual features and targets for the random material
random_material_features = X_test_scaled.iloc[[random_index]]
random_material_actual_targets = Y_test.iloc[[random_index]]

# Get the original material id and formula
original_material_id = df_engineered.loc[random_material_features.index[0], 'material_id']
original_formula_pretty = df_engineered.loc[random_material_features.index[0], 'formula_pretty']

# Predict the targets for the random material
random_material_predicted_targets_scaled = multioutput_xgb_final.predict(random_material_features)

# Since the model was trained on scaled features, the predictions are directly comparable
# to the scaled Y values if Y was scaled. However, Y was NOT scaled in this notebook.
# So, the predictions are in the original scale of the target variables.

print(f"\nTesting prediction on a random material (Index: {random_index}):")
print(f"  Material ID: {original_material_id}")
print(f"  Formula: {original_formula_pretty}")

print("\nActual Target Values:")
# Add units to the column names for actual targets
actual_targets_with_units = random_material_actual_targets.copy()
actual_targets_with_units.columns = ['theoretical_capacity_grav (mAh/g)', 'average_voltage (V)', 'energy_above_hull (eV/atom)']
print(actual_targets_with_units.to_markdown(numalign="left", stralign="left"))


print("\nPredicted Target Values:")
# Convert the numpy array prediction to a pandas DataFrame for easier printing
predicted_df = pd.DataFrame(random_material_predicted_targets_scaled, columns=Y_cols, index=[0])
# Add units to the column names for predicted targets
predicted_df.columns = ['theoretical_capacity_grav (mAh/g)', 'average_voltage (V)', 'energy_above_hull (eV/atom)']
print(predicted_df.to_markdown(numalign="left", stralign="left"))

# Calculate absolute errors for this specific material
abs_errors = np.abs(random_material_actual_targets.values - random_material_predicted_targets_scaled)
# Add units to the column names for absolute errors
abs_errors_df = pd.DataFrame(abs_errors, columns=[f'Absolute Error ({col})' for col in ['theoretical_capacity_grav (mAh/g)', 'average_voltage (V)', 'energy_above_hull (eV/atom)']], index=[0])

print("\nAbsolute Errors for this Prediction:")
print(abs_errors_df.to_markdown(numalign="left", stralign="left"))

# Calculate relative errors for this specific material
# Avoid division by zero for actual values close to zero
relative_errors = np.where(random_material_actual_targets.values != 0,
                           abs_errors / random_material_actual_targets.values,
                           np.nan) # Use NaN where actual value is 0

# Add units to the column names for relative errors
relative_errors_df = pd.DataFrame(relative_errors, columns=[f'Relative Error ({col})' for col in ['theoretical_capacity_grav (%)', 'average_voltage (%)', 'energy_above_hull (%)']], index=[0])

print("\nRelative Errors for this Prediction (as %):")
# Convert relative error to percentage for display
relative_errors_df = relative_errors_df * 100
print(relative_errors_df.to_markdown(numalign="left", stralign="left"))


Testing prediction on a random material (Index: 539):
  Material ID: mp-758020
  Formula: LiCu3(CO3)3

Actual Target Values:
|      | theoretical_capacity_grav (mAh/g)   | average_voltage (V)   | energy_above_hull (eV/atom)   |
|:-----|:------------------------------------|:----------------------|:------------------------------|
| 2210 | 136.921                             | 2.44149               | 0.110563                      |

Predicted Target Values:
|    | theoretical_capacity_grav (mAh/g)   | average_voltage (V)   | energy_above_hull (eV/atom)   |
|:---|:------------------------------------|:----------------------|:------------------------------|
| 0  | 154.749                             | 2.35688               | 0.0890837                     |

Absolute Errors for this Prediction:
|    | Absolute Error (theoretical_capacity_grav (mAh/g))   | Absolute Error (average_voltage (V))   | Absolute Error (energy_above_hull (eV/atom))   |
|:---|:---------------------------------------

## Hyperparameter Tuning for XGBoost Model

In [ ]:
# Define the base XGBoost model
xgb_base_model_tuned = xgb.XGBRegressor(
    objective='reg:squarederror', # Standard for regression
    random_state=9,
    n_jobs=-1, # Use all available CPU cores
    tree_method='hist' # Often faster for larger datasets
)

In [ ]:
# Wrap it in MultiOutputRegressor
multioutput_xgb_tuned = MultiOutputRegressor(xgb_base_model_tuned)

# Define the parameter distribution for RandomizedSearchCV for XGBoost
param_distributions_xgb = {
    'estimator__n_estimators': [100, 200, 300, 400, 500, 600], # Number of boosting rounds (trees)
    'estimator__learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2], # Step size shrinkage
    'estimator__max_depth': [3, 4, 5, 6, 7, 8, 9, 10], # Maximum depth of a tree
    'estimator__subsample': [0.6, 0.7, 0.8, 0.9, 1.0], # Subsample ratio of the training instance
    'estimator__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0], # Subsample ratio of columns when constructing each tree
    'estimator__gamma': [0, 0.1, 0.2, 0.3, 0.4], # Minimum loss reduction required to make a further partition on a leaf node
    'estimator__reg_alpha': [0, 0.005, 0.01, 0.05, 0.1, 0.5, 1], # L1 regularization term on weights
    'estimator__reg_lambda': [0, 0.005, 0.01, 0.05, 0.1, 0.5, 1] # L2 regularization term on weights
}


In [ ]:
# Initialize RandomizedSearchCV
random_search_xgb = RandomizedSearchCV(
    estimator=multioutput_xgb_tuned,
    param_distributions=param_distributions_xgb,
    n_iter=20, # Increased n_iter for a more thorough search
    cv=5,
    verbose=2, # Set to 1 or 2 for more detailed output during search
    random_state=9,
    n_jobs=-1 # Use all available CPU cores
)

# Fit RandomizedSearchCV to the scaled training data and targets
random_search_xgb.fit(X_train_scaled, Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=5,
                   estimator=MultiOutputRegressor(estimator=XGBRegressor(base_score=None,
                                                                         booster=None,
                                                                         callbacks=None,
                                                                         colsample_bylevel=None,
                                                                         colsample_bynode=None,
                                                                         colsample_bytree=None,
                                                                         device=None,
                                                                         early_stopping_rounds=None,
                                                                         enable_categorical=False,
                                                                         eval_metric=None,
                                                                         feature_types=None,
                                                                         gamma=None,
                                                                         grow_policy=None,
                                                                         importance_type=None,
                                                                         interaction_...
                                        'estimator__gamma': [0, 0.1, 0.2, 0.3,
                                                             0.4],
                                        'estimator__learning_rate': [0.01, 0.05,
                                                                     0.1, 0.15,
                                                                     0.2],
                                        'estimator__max_depth': [3, 4, 5, 6, 7,
                                                                 8, 9, 10],
                                        'estimator__n_estimators': [100, 200,
                                                                    300, 400,
                                                                    500, 600],
                                        'estimator__reg_alpha': [0, 0.005, 0.01,
                                                                 0.05, 0.1, 0.5,
                                                                 1],
                                        'estimator__reg_lambda': [0, 0.005,
                                                                  0.01, 0.05,
                                                                  0.1, 0.5, 1],
                                        'estimator__subsample': [0.6, 0.7, 0.8,
                                                                 0.9, 1.0]},
                   random_state=9, verbose=2)

In [ ]:
# Get the best parameters and the best score
best_params_xgb = random_search_xgb.best_params_
best_score_xgb = random_search_xgb.best_score_

print(f"Best parameters found for XGBoost: {best_params_xgb}")
print(f"Best score found for XGBoost: {best_score_xgb}")

Best parameters found for XGBoost: {'estimator__subsample': 0.8, 'estimator__reg_lambda': 1, 'estimator__reg_alpha': 0, 'estimator__n_estimators': 200, 'estimator__max_depth': 8, 'estimator__learning_rate': 0.15, 'estimator__gamma': 0, 'estimator__colsample_bytree': 1.0}
Best score found for XGBoost: 0.6646795511245728


In [ ]:
# Re-evaluate the best model with cross-validation on all targets for robust metrics

# Get the best estimator from the search
best_multioutput_xgb = random_search_xgb.best_estimator_

# Prepare lists to store scores for each target across all folds
xgb_tuned_target_r2_scores = {col: [] for col in Y_train.columns}
xgb_tuned_target_mae_scores = {col: [] for col in Y_train.columns}
xgb_tuned_target_rmse_scores = {col: [] for col in Y_train.columns}


In [ ]:
# Perform cross-validation with the best estimator
kfold = KFold(n_splits=5, shuffle=True, random_state=9)

for fold_idx, (train_index, test_index) in enumerate(kfold.split(X_train_scaled, Y_train)):
    X_train_fold, X_test_fold = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]

    # Fit a new instance of the best model for each fold (important for proper CV)
    # Extract the base estimator parameters from best_params_xgb
    fold_xgb_base_model_tuned = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=best_params_xgb['estimator__n_estimators'],
        learning_rate=best_params_xgb['estimator__learning_rate'],
        max_depth=best_params_xgb['estimator__max_depth'],
        subsample=best_params_xgb['estimator__subsample'],
        colsample_bytree=best_params_xgb['estimator__colsample_bytree'],
        gamma=best_params_xgb['estimator__gamma'],
        reg_alpha=best_params_xgb['estimator__reg_alpha'],
        reg_lambda=best_params_xgb['estimator__reg_lambda'],
        random_state=9,
        n_jobs=-1,
        tree_method='hist'
    )
    fold_multioutput_xgb_tuned = MultiOutputRegressor(fold_xgb_base_model_tuned)
    fold_multioutput_xgb_tuned.fit(X_train_fold, Y_train_fold)

    Y_pred_fold = fold_multioutput_xgb_tuned.predict(X_test_fold)
    Y_pred_fold_df = pd.DataFrame(Y_pred_fold, columns=Y_test_fold.columns, index=Y_test_fold.index)

    for i, target_col in enumerate(Y_test_fold.columns):
        r2 = r2_score(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        mae = mean_absolute_error(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        rmse = np.sqrt(mean_squared_error(Y_test_fold[target_col], Y_pred_fold_df[target_col]))

        xgb_tuned_target_r2_scores[target_col].append(r2)
        xgb_tuned_target_mae_scores[target_col].append(mae)
        xgb_tuned_target_rmse_scores[target_col].append(rmse)

In [ ]:
# Print aggregated cross-validation results for the tuned XGBoost
print("\n--- Tuned XGBoost Cross-Validation Results (Mean across 5 folds, Per Target) ---")
for target_col in Y_train.columns:
    print(f"\nMetrics for Target: {target_col}")
    print(f"  Mean R-squared (R²): {np.mean(xgb_tuned_target_r2_scores[target_col]):.4f} (Std Dev: {np.std(xgb_tuned_target_r2_scores[target_col]):.4f})")
    print(f"  Mean MAE: {np.mean(xgb_tuned_target_mae_scores[target_col]):.4f} (Std Dev: {np.std(xgb_tuned_target_mae_scores[target_col]):.4f})")
    print(f"  Mean RMSE: {np.mean(xgb_tuned_target_rmse_scores[target_col]):.4f} (Std Dev: {np.std(xgb_tuned_target_rmse_scores[target_col]):.4f})")



--- Tuned XGBoost Cross-Validation Results (Mean across 5 folds, Per Target) ---

Metrics for Target: theoretical_capacity_grav
  Mean R-squared (R²): 0.8705 (Std Dev: 0.0248)
  Mean MAE: 19.9334 (Std Dev: 1.4438)
  Mean RMSE: 52.2823 (Std Dev: 12.1327)

Metrics for Target: average_voltage
  Mean R-squared (R²): 0.8067 (Std Dev: 0.1540)
  Mean MAE: 0.4362 (Std Dev: 0.1741)
  Mean RMSE: 2.8908 (Std Dev: 3.6284)

Metrics for Target: energy_above_hull
  Mean R-squared (R²): 0.4592 (Std Dev: 0.0350)
  Mean MAE: 0.0246 (Std Dev: 0.0009)
  Mean RMSE: 0.0333 (Std Dev: 0.0011)


In [ ]:
# Final Evaluation on the untouched Test Set with the best tuned model
print("\n--- Final Evaluation of Tuned XGBoost Model on Test Set ---")
Y_pred_tuned_xgb_test = best_multioutput_xgb.predict(X_test_scaled)
Y_pred_tuned_xgb_test_df = pd.DataFrame(Y_pred_tuned_xgb_test, columns=Y_test.columns, index=Y_test.index)

for i, target_col in enumerate(Y_test.columns):
    r2 = r2_score(Y_test[target_col], Y_pred_tuned_xgb_test_df[target_col])
    mae = mean_absolute_error(Y_test[target_col], Y_pred_tuned_xgb_test_df[target_col])
    rmse = np.sqrt(mean_squared_error(Y_test[target_col], Y_pred_tuned_xgb_test_df[target_col]))

    print(f"\nMetrics for Target: {target_col} (Test Set, Tuned XGBoost)")
    print(f"  R-squared (R²): {r2:.4f}")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")


--- Final Evaluation of Tuned XGBoost Model on Test Set ---

Metrics for Target: theoretical_capacity_grav (Test Set, Tuned XGBoost)
  R-squared (R²): 0.6534
  Mean Absolute Error (MAE): 18.2208
  Root Mean Squared Error (RMSE): 42.0546

Metrics for Target: average_voltage (Test Set, Tuned XGBoost)
  R-squared (R²): 0.8449
  Mean Absolute Error (MAE): 0.3676
  Root Mean Squared Error (RMSE): 1.5609

Metrics for Target: energy_above_hull (Test Set, Tuned XGBoost)
  R-squared (R²): 0.5002
  Mean Absolute Error (MAE): 0.0236
  Root Mean Squared Error (RMSE): 0.0326


## Testing on dataset using tuned model

In [ ]:
# Select a random index from the test set
random_index = random.randint(0, len(X_test_scaled) - 1)

# Get the actual features and targets for the random material
random_material_features = X_test_scaled.iloc[[random_index]]
random_material_actual_targets = Y_test.iloc[[random_index]]

# Get the original material id and formula
original_material_id = df_engineered.loc[random_material_features.index[0], 'material_id']
original_formula_pretty = df_engineered.loc[random_material_features.index[0], 'formula_pretty']

# Predict the targets for the random material using the best tuned model
random_material_predicted_targets_scaled_tuned = best_multioutput_xgb.predict(random_material_features)


print(f"\nTesting prediction on a random material using the TUNED model (Index: {random_index}):")
print(f"  Material ID: {original_material_id}")
print(f"  Formula: {original_formula_pretty}")

print("\nActual Target Values:")
# Add units to the column names for actual targets
actual_targets_with_units = random_material_actual_targets.copy()
actual_targets_with_units.columns = ['theoretical_capacity_grav (mAh/g)', 'average_voltage (V)', 'energy_above_hull (eV/atom)']
print(actual_targets_with_units.to_markdown(numalign="left", stralign="left"))


print("\nPredicted Target Values (Tuned Model):")
# Convert the numpy array prediction to a pandas DataFrame for easier printing
predicted_df_tuned = pd.DataFrame(random_material_predicted_targets_scaled_tuned, columns=Y_cols, index=[0])
# Add units to the column names for predicted targets
predicted_df_tuned.columns = ['theoretical_capacity_grav (mAh/g)', 'average_voltage (V)', 'energy_above_hull (eV/atom)']
print(predicted_df_tuned.to_markdown(numalign="left", stralign="left"))

# Calculate absolute errors for this specific material using the tuned model
abs_errors_tuned = np.abs(random_material_actual_targets.values - random_material_predicted_targets_scaled_tuned)
# Add units to the column names for absolute errors
abs_errors_tuned_df = pd.DataFrame(abs_errors_tuned, columns=[f'Absolute Error ({col})' for col in ['theoretical_capacity_grav (mAh/g)', 'average_voltage (V)', 'energy_above_hull (eV/atom)']], index=[0])

print("\nAbsolute Errors for this Prediction (Tuned Model):")
print(abs_errors_tuned_df.to_markdown(numalign="left", stralign="left"))

# Calculate relative errors for this specific material using the tuned model
# Avoid division by zero for actual values close to zero
relative_errors_tuned = np.where(random_material_actual_targets.values != 0,
                                 abs_errors_tuned / random_material_actual_targets.values,
                                 np.nan) # Use NaN where actual value is 0

# Add units to the column names for relative errors
relative_errors_tuned_df = pd.DataFrame(relative_errors_tuned, columns=[f'Relative Error ({col})' for col in ['theoretical_capacity_grav (%)', 'average_voltage (%)', 'energy_above_hull (%)']], index=[0])

print("\nRelative Errors for this Prediction (Tuned Model, as %):")
# Convert relative error to percentage for display
relative_errors_tuned_df = relative_errors_tuned_df * 100
print(relative_errors_tuned_df.to_markdown(numalign="left", stralign="left"))


Testing prediction on a random material using the TUNED model (Index: 343):
  Material ID: mp-32023
  Formula: LiMn2(PO4)3

Actual Target Values:
|      | theoretical_capacity_grav (mAh/g)   | average_voltage (V)   | energy_above_hull (eV/atom)   |
|:-----|:------------------------------------|:----------------------|:------------------------------|
| 1175 | 128.973                             | 4.01943               | 0.0619951                     |

Predicted Target Values (Tuned Model):
|    | theoretical_capacity_grav (mAh/g)   | average_voltage (V)   | energy_above_hull (eV/atom)   |
|:---|:------------------------------------|:----------------------|:------------------------------|
| 0  | 160.25                              | 3.91649               | 0.0733415                     |

Absolute Errors for this Prediction (Tuned Model):
|    | Absolute Error (theoretical_capacity_grav (mAh/g))   | Absolute Error (average_voltage (V))   | Absolute Error (energy_above_hull (eV/atom))  

## Save model and scaler

In [ ]:
# Define the path for the models folder
model_folder_path = '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model'

model_filename = os.path.join(model_folder_path, "tuned_xgboost_multi_output_model_v2.joblib")
scaler_filename = os.path.join(model_folder_path, "scaler_X_v2.joblib")
feature_names_filename = os.path.join(model_folder_path, "feature_column_names_v2.joblib")

try:
    joblib.dump(best_multioutput_xgb, model_filename)
    joblib.dump(scaler_X, scaler_filename)
    joblib.dump(X_train_scaled.columns.tolist(), feature_names_filename)

    print(f"Tuned XGBoost model saved to: '{model_filename}'")
    print(f"Scaler saved to: '{scaler_filename}'")
    print(f"Feature column names saved to: '{feature_names_filename}'")
except Exception as e:
    print(f"Error saving model or scaler: {e}")

Tuned XGBoost model saved to: '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/tuned_xgboost_multi_output_model_v2.joblib'
Scaler saved to: '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/scaler_X_v2.joblib'
Feature column names saved to: '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/feature_column_names_v2.joblib'


In [ ]:
# Define the path for saving the training data
data_output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/Training_Data'
os.makedirs(data_output_folder_path, exist_ok=True) # Create the directory if it doesn't exist

# Define filenames for the training data
X_train_filename = os.path.join(data_output_folder_path, "X_train_scaled.csv")
Y_train_filename = os.path.join(data_output_folder_path, "Y_train.csv")
X_test_filename = os.path.join(data_output_folder_path, "X_test_scaled.csv")
Y_test_filename = os.path.join(data_output_folder_path, "Y_test.csv")

try:
    # Add 'material_id' and 'formula_pretty' back to the scaled dataframes using their index
    X_train_scaled_with_ids = X_train_scaled.copy()
    X_test_scaled_with_ids = X_test_scaled.copy()

    # Ensure the index is aligned with the original dataframe to get the correct IDs and formulas
    X_train_scaled_with_ids[['material_id', 'formula_pretty']] = df_engineered.loc[X_train_scaled_with_ids.index, ['material_id', 'formula_pretty']]
    X_test_scaled_with_ids[['material_id', 'formula_pretty']] = df_engineered.loc[X_test_scaled_with_ids.index, ['material_id', 'formula_pretty']]


    # Save the training and test features (scaled) with IDs and formulas
    X_train_scaled_with_ids.to_csv(X_train_filename, index=True) # Keep index to link back if needed
    X_test_scaled_with_ids.to_csv(X_test_filename, index=True)   # Keep index to link back if needed

    # Save the training and test targets
    Y_train.to_csv(Y_train_filename, index=True) # Keep index to link back if needed
    Y_test.to_csv(Y_test_filename, index=True)   # Keep index to link back if needed

    print(f"Training data (X_train_scaled_with_ids and Y_train) saved to: '{data_output_folder_path}'")
    print(f"Test data (X_test_scaled_with_ids and Y_test) saved to: '{data_output_folder_path}'")

except Exception as e:
    print(f"Error saving training data: {e}")

Training data (X_train_scaled_with_ids and Y_train) saved to: '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/Training_Data'
Test data (X_test_scaled_with_ids and Y_test) saved to: '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/Training_Data'


# Random Forest

In [ ]:
# Define the base Random Forest model
rf_base_model = RandomForestRegressor(
    n_estimators=300,  # Number of trees in the forest
    max_depth=10,      # Maximum depth of the trees
    min_samples_split=5, # Minimum number of samples required to split an internal node
    min_samples_leaf=3,  # Minimum number of samples required to be at a leaf node
    random_state=9,
    n_jobs=-1          # Use all available CPU cores
)

# Wrap the Random Forest model in MultiOutputRegressor
multioutput_rf = MultiOutputRegressor(rf_base_model)

# Prepare lists to store scores for each target across all folds
rf_target_r2_scores = {col: [] for col in Y_train.columns}
rf_target_mae_scores = {col: [] for col in Y_train.columns}
rf_target_rmse_scores = {col: [] for col in Y_train.columns}

# Perform cross-validation with the Random Forest model
kfold = KFold(n_splits=5, shuffle=True, random_state=9)

print("\n--- Random Forest Cross-Validation ---")
for fold_idx, (train_index, test_index) in enumerate(kfold.split(X_train_scaled, Y_train)):
    X_train_fold, X_test_fold = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]

    print(f"Fold {fold_idx + 1}/5: Training...")
    # Fit a new instance of the MultiOutput Random Forest model for each fold
    fold_multioutput_rf = MultiOutputRegressor(rf_base_model)
    fold_multioutput_rf.fit(X_train_fold, Y_train_fold)

    Y_pred_fold = fold_multioutput_rf.predict(X_test_fold)
    # Convert predictions back to DataFrame with column names for clarity
    Y_pred_fold_df = pd.DataFrame(Y_pred_fold, columns=Y_test_fold.columns, index=Y_test_fold.index)

    print(f"Fold {fold_idx + 1}/5: Evaluating...")
    for i, target_col in enumerate(Y_test_fold.columns):
        r2 = r2_score(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        mae = mean_absolute_error(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        rmse = np.sqrt(mean_squared_error(Y_test_fold[target_col], Y_pred_fold_df[target_col]))

        rf_target_r2_scores[target_col].append(r2)
        rf_target_mae_scores[target_col].append(mae)
        rf_target_rmse_scores[target_col].append(rmse)


--- Random Forest Cross-Validation ---
Fold 1/5: Training...
Fold 1/5: Evaluating...
Fold 2/5: Training...
Fold 2/5: Evaluating...
Fold 3/5: Training...
Fold 3/5: Evaluating...
Fold 4/5: Training...
Fold 4/5: Evaluating...
Fold 5/5: Training...
Fold 5/5: Evaluating...


In [ ]:
# Print aggregated cross-validation results for Random Forest
print("\n Random Forest Cross-Validation Results (Mean across 5 folds, Per Target)")
for target_col in Y_train.columns:
    print(f"\nMetrics for Target: {target_col}")
    print(f"  Mean R-squared (R²): {np.mean(rf_target_r2_scores[target_col]):.4f} (Std Dev: {np.std(rf_target_r2_scores[target_col]):.4f})")
    print(f"  Mean MAE: {np.mean(rf_target_mae_scores[target_col]):.4f} (Std Dev: {np.std(rf_target_mae_scores[target_col]):.4f})")
    print(f"  Mean RMSE: {np.mean(rf_target_rmse_scores[target_col]):.4f} (Std Dev: {np.std(rf_target_rmse_scores[target_col]):.4f})")





 Random Forest Cross-Validation Results (Mean across 5 folds, Per Target)

Metrics for Target: theoretical_capacity_grav
  Mean R-squared (R²): 0.8070 (Std Dev: 0.0544)
  Mean MAE: 25.4524 (Std Dev: 1.6048)
  Mean RMSE: 62.9646 (Std Dev: 15.4942)

Metrics for Target: average_voltage
  Mean R-squared (R²): 0.7313 (Std Dev: 0.2368)
  Mean MAE: 0.5274 (Std Dev: 0.2312)
  Mean RMSE: 3.4047 (Std Dev: 4.4331)

Metrics for Target: energy_above_hull
  Mean R-squared (R²): 0.3983 (Std Dev: 0.0159)
  Mean MAE: 0.0268 (Std Dev: 0.0004)
  Mean RMSE: 0.0351 (Std Dev: 0.0005)


## Testing Random forest model on test set

In [ ]:
# Train on the full training set and evaluate on test set for a final view
print("\n Random Forest Model Evaluation on Test Set")
multioutput_rf_final = MultiOutputRegressor(rf_base_model)
multioutput_rf_final.fit(X_train_scaled, Y_train) # Train on full training data

Y_pred_rf_test = multioutput_rf_final.predict(X_test_scaled)
Y_pred_rf_test_df = pd.DataFrame(Y_pred_rf_test, columns=Y_test.columns, index=Y_test.index)

for i, target_col in enumerate(Y_test.columns):
    r2 = r2_score(Y_test[target_col], Y_pred_rf_test_df[target_col])
    mae = mean_absolute_error(Y_test[target_col], Y_pred_rf_test_df[target_col])
    rmse = np.sqrt(mean_squared_error(Y_test[target_col], Y_pred_rf_test_df[target_col]))

    print(f"\nMetrics for Target: {target_col} (Test Set)")
    print(f"  R-squared (R²): {r2:.4f}")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")


 Random Forest Model Evaluation on Test Set

Metrics for Target: theoretical_capacity_grav (Test Set)
  R-squared (R²): 0.5862
  Mean Absolute Error (MAE): 22.4741
  Root Mean Squared Error (RMSE): 45.9480

Metrics for Target: average_voltage (Test Set)
  R-squared (R²): 0.7463
  Mean Absolute Error (MAE): 0.4692
  Root Mean Squared Error (RMSE): 1.9962

Metrics for Target: energy_above_hull (Test Set)
  R-squared (R²): 0.4107
  Mean Absolute Error (MAE): 0.0263
  Root Mean Squared Error (RMSE): 0.0354


## Hyperparameter tuning on Random Forest model

In [ ]:
# Define the base Random Forest model for tuning
rf_base_model_tuned = RandomForestRegressor(
    random_state=9,
    n_jobs=-1 # Use all available CPU cores
)

# Wrap it in MultiOutputRegressor
multioutput_rf_tuned = MultiOutputRegressor(rf_base_model_tuned)

# Define the parameter distribution for RandomizedSearchCV for Random Forest
param_distributions_rf = {
    'estimator__n_estimators': [100, 200, 300], # Number of trees
    'estimator__max_depth': [5, 10, 15], # Maximum depth or None for full growth
    'estimator__min_samples_split': [2, 5], # Minimum samples to split a node
    'estimator__min_samples_leaf': [1, 2], # Minimum samples at a leaf node
    'estimator__bootstrap': [True, False] # Whether bootstrap samples are used
}

# Initialize RandomizedSearchCV for Random Forest
random_search_rf = RandomizedSearchCV(
    estimator=multioutput_rf_tuned,
    param_distributions=param_distributions_rf,
    n_iter=25, # Number of parameter settings that are sampled
    cv=4,      # Number of folds in cross-validation
    verbose=2,
    random_state=9,
    n_jobs=-1  # Use all available CPU cores
)

# Fit RandomizedSearchCV to the scaled training data and targets
print("\n--- Performing Randomized Search for Random Forest Hyperparameter Tuning ---")
random_search_rf.fit(X_train_scaled, Y_train)

# Get the best parameters and the best score
best_params_rf = random_search_rf.best_params_
best_score_rf = random_search_rf.best_score_ # This is the mean score of the best estimator on the held out data.

print(f"\nBest parameters found for Random Forest: {best_params_rf}")
print(f"Best score found for Random Forest (mean CV score): {best_score_rf:.4f}")

# Re-evaluate the best model with cross-validation on all targets for robust metrics

# Get the best estimator from the search
best_multioutput_rf = random_search_rf.best_estimator_

# Prepare lists to store scores for each target across all folds
rf_tuned_target_r2_scores = {col: [] for col in Y_train.columns}
rf_tuned_target_mae_scores = {col: [] for col in Y_train.columns}
rf_tuned_target_rmse_scores = {col: [] for col in Y_train.columns}

# Perform cross-validation with the best estimator
kfold = KFold(n_splits=5, shuffle=True, random_state=9)

print("\n--- Cross-Validation Results for Tuned Random Forest ---")
for fold_idx, (train_index, test_index) in enumerate(kfold.split(X_train_scaled, Y_train)):
    X_train_fold, X_test_fold = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]

    print(f"Fold {fold_idx + 1}/5: Training tuned model...")
    # Fit a new instance of the best model for each fold (important for proper CV)
    fold_rf_base_model_tuned = RandomForestRegressor(
        n_estimators=best_params_rf['estimator__n_estimators'],
        max_depth=best_params_rf['estimator__max_depth'],
        min_samples_split=best_params_rf['estimator__min_samples_split'],
        min_samples_leaf=best_params_rf['estimator__min_samples_leaf'],
        bootstrap=best_params_rf['estimator__bootstrap'],
        random_state=9,
        n_jobs=-1
    )
    fold_multioutput_rf_tuned = MultiOutputRegressor(fold_rf_base_model_tuned)
    fold_multioutput_rf_tuned.fit(X_train_fold, Y_train_fold)

    Y_pred_fold = fold_multioutput_rf_tuned.predict(X_test_fold)
    Y_pred_fold_df = pd.DataFrame(Y_pred_fold, columns=Y_test_fold.columns, index=Y_test_fold.index)

    print(f"Fold {fold_idx + 1}/5: Evaluating tuned model...")
    for i, target_col in enumerate(Y_test_fold.columns):
        r2 = r2_score(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        mae = mean_absolute_error(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        rmse = np.sqrt(mean_squared_error(Y_test_fold[target_col], Y_pred_fold_df[target_col]))

        rf_tuned_target_r2_scores[target_col].append(r2)
        rf_tuned_target_mae_scores[target_col].append(mae)
        rf_tuned_target_rmse_scores[target_col].append(rmse)




--- Performing Randomized Search for Random Forest Hyperparameter Tuning ---
Fitting 4 folds for each of 25 candidates, totalling 100 fits

Best parameters found for Random Forest: {'estimator__n_estimators': 200, 'estimator__min_samples_split': 2, 'estimator__min_samples_leaf': 1, 'estimator__max_depth': 15, 'estimator__bootstrap': True}
Best score found for Random Forest (mean CV score): 0.6524

--- Cross-Validation Results for Tuned Random Forest ---
Fold 1/5: Training tuned model...
Fold 1/5: Evaluating tuned model...
Fold 2/5: Training tuned model...
Fold 2/5: Evaluating tuned model...
Fold 3/5: Training tuned model...
Fold 3/5: Evaluating tuned model...
Fold 4/5: Training tuned model...
Fold 4/5: Evaluating tuned model...
Fold 5/5: Training tuned model...
Fold 5/5: Evaluating tuned model...


In [ ]:
# Print aggregated cross-validation results for the tuned Random Forest
print("\n--- Tuned Random Forest Cross-Validation Results (Mean across 5 folds, Per Target) ---")
for target_col in Y_train.columns:
    print(f"\nMetrics for Target: {target_col}")
    print(f"  Mean R-squared (R²): {np.mean(rf_tuned_target_r2_scores[target_col]):.4f} (Std Dev: {np.std(rf_tuned_target_r2_scores[target_col]):.4f})")
    print(f"  Mean MAE: {np.mean(rf_tuned_target_mae_scores[target_col]):.4f} (Std Dev: {np.std(rf_tuned_target_mae_scores[target_col]):.4f})")
    print(f"  Mean RMSE: {np.mean(rf_tuned_target_rmse_scores[target_col]):.4f} (Std Dev: {np.std(rf_tuned_target_rmse_scores[target_col]):.4f})")




--- Tuned Random Forest Cross-Validation Results (Mean across 5 folds, Per Target) ---

Metrics for Target: theoretical_capacity_grav
  Mean R-squared (R²): 0.8114 (Std Dev: 0.0472)
  Mean MAE: 22.7258 (Std Dev: 1.6363)
  Mean RMSE: 62.5175 (Std Dev: 15.0600)

Metrics for Target: average_voltage
  Mean R-squared (R²): 0.7934 (Std Dev: 0.1878)
  Mean MAE: 0.4743 (Std Dev: 0.1984)
  Mean RMSE: 3.0983 (Std Dev: 4.0410)

Metrics for Target: energy_above_hull
  Mean R-squared (R²): 0.4366 (Std Dev: 0.0128)
  Mean MAE: 0.0256 (Std Dev: 0.0004)
  Mean RMSE: 0.0340 (Std Dev: 0.0005)


## Testing tuned random forest model on test set

In [ ]:
# Final Evaluation on the untouched Test Set with the best tuned model
print("\n--- Final Evaluation of Tuned Random Forest Model on Test Set ---")
Y_pred_tuned_rf_test = best_multioutput_rf.predict(X_test_scaled)
Y_pred_tuned_rf_test_df = pd.DataFrame(Y_pred_tuned_rf_test, columns=Y_test.columns, index=Y_test.index)

for i, target_col in enumerate(Y_test.columns):
    r2 = r2_score(Y_test[target_col], Y_pred_tuned_rf_test_df[target_col])
    mae = mean_absolute_error(Y_test[target_col], Y_pred_tuned_rf_test_df[target_col])
    rmse = np.sqrt(mean_squared_error(Y_test[target_col], Y_pred_tuned_rf_test_df[target_col]))

    print(f"\nMetrics for Target: {target_col} (Test Set, Tuned Random Forest)")
    print(f"  R-squared (R²): {r2:.4f}")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")


--- Final Evaluation of Tuned Random Forest Model on Test Set ---

Metrics for Target: theoretical_capacity_grav (Test Set, Tuned Random Forest)
  R-squared (R²): 0.6406
  Mean Absolute Error (MAE): 19.0836
  Root Mean Squared Error (RMSE): 42.8224

Metrics for Target: average_voltage (Test Set, Tuned Random Forest)
  R-squared (R²): 0.8924
  Mean Absolute Error (MAE): 0.3814
  Root Mean Squared Error (RMSE): 1.3002

Metrics for Target: energy_above_hull (Test Set, Tuned Random Forest)
  R-squared (R²): 0.4496
  Mean Absolute Error (MAE): 0.0251
  Root Mean Squared Error (RMSE): 0.0342


## Save Tuned Random Forest model

In [ ]:
# Define the path for the models folder
model_folder_path = '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model'
os.makedirs(model_folder_path, exist_ok=True) # Create the directory if it doesn't exist

# Define the filename for the tuned Random Forest model
rf_model_filename_tuned = os.path.join(model_folder_path, "tuned_random_forest_multi_output_model_v2.joblib")

try:
    # Save the best tuned Random Forest model
    joblib.dump(best_multioutput_rf, rf_model_filename_tuned)

    print(f"Tuned Random Forest model saved to: '{rf_model_filename_tuned}'")

except Exception as e:
    print(f"Error saving tuned Random Forest model: {e}")


Tuned Random Forest model saved to: '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/tuned_random_forest_multi_output_model_v2.joblib'


# LightGBM

In [ ]:
# Define the base LightGBM model
lgb_base_model = lgb.LGBMRegressor(
    objective='regression_l1', # MAE objective
    metric='mae',              # Metric for evaluation during training
    n_estimators=300,          # A good starting point for boosting rounds
    learning_rate=0.1,         # Step size shrinkage
    num_leaves=31,             # Maximum number of leaves in one tree
    max_depth=-1,              # No limit on tree depth
    min_child_samples=20,      # Minimum number of data needed in a child
    subsample=0.8,             # Fraction of samples for fitting the individual base learners
    colsample_bytree=0.8,      # Fraction of features for fitting the individual base learners
    random_state=9,
    n_jobs=-1,
    boosting_type='gbdt'       # Traditional Gradient Boosting Decision Tree
)

# Wrap the LightGBM model in MultiOutputRegressor
multioutput_lgb = MultiOutputRegressor(lgb_base_model)

# Prepare lists to store scores for each target across all folds
lgb_target_r2_scores = {col: [] for col in Y_train.columns}
lgb_target_mae_scores = {col: [] for col in Y_train.columns}
lgb_target_rmse_scores = {col: [] for col in Y_train.columns}

# Perform cross-validation with the LightGBM model
kfold = KFold(n_splits=5, shuffle=True, random_state=9)

print("\n--- LightGBM Cross-Validation ---")
for fold_idx, (train_index, test_index) in enumerate(kfold.split(X_train_scaled, Y_train)):
    X_train_fold, X_test_fold = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]

    print(f"Fold {fold_idx + 1}/5: Training...")
    # Fit a new instance of the MultiOutput LightGBM model for each fold
    fold_multioutput_lgb = MultiOutputRegressor(lgb_base_model)
    fold_multioutput_lgb.fit(X_train_fold, Y_train_fold)

    Y_pred_fold = fold_multioutput_lgb.predict(X_test_fold)
    # Convert predictions back to DataFrame with column names for clarity
    Y_pred_fold_df = pd.DataFrame(Y_pred_fold, columns=Y_test_fold.columns, index=Y_test_fold.index)

    print(f"Fold {fold_idx + 1}/5: Evaluating...")
    for i, target_col in enumerate(Y_test_fold.columns):
        r2 = r2_score(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        mae = mean_absolute_error(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        rmse = np.sqrt(mean_squared_error(Y_test_fold[target_col], Y_pred_fold_df[target_col]))

        lgb_target_r2_scores[target_col].append(r2)
        lgb_target_mae_scores[target_col].append(mae)
        lgb_target_rmse_scores[target_col].append(rmse)




--- LightGBM Cross-Validation ---
Fold 1/5: Training...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001089 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4415
[LightGBM] [Info] Number of data points in the train set: 2204, number of used features: 49
[LightGBM] [Info] Start training from score 111.869675
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001079 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4415
[LightGBM] [Info] Number of data points in the train set: 2204, number of used features: 49
[LightGBM] [Info] Start training from score 2.491324
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4415
[LightGBM] [Info] Number of data points in the train set: 2204,

In [ ]:
# Print aggregated cross-validation results for LightGBM
print("\n LightGBM Cross-Validation Results (Mean across 5 folds, Per Target)")
for target_col in Y_train.columns:
    print(f"\nMetrics for Target: {target_col}")
    print(f"  Mean R-squared (R²): {np.mean(lgb_target_r2_scores[target_col]):.4f} (Std Dev: {np.std(lgb_target_r2_scores[target_col]):.4f})")
    print(f"  Mean MAE: {np.mean(lgb_target_mae_scores[target_col]):.4f} (Std Dev: {np.std(lgb_target_mae_scores[target_col]):.4f})")
    print(f"  Mean RMSE: {np.mean(lgb_target_rmse_scores[target_col]):.4f} (Std Dev: {np.std(lgb_target_rmse_scores[target_col]):.4f})")




 LightGBM Cross-Validation Results (Mean across 5 folds, Per Target)

Metrics for Target: theoretical_capacity_grav
  Mean R-squared (R²): 0.6299 (Std Dev: 0.0792)
  Mean MAE: 26.0190 (Std Dev: 3.5175)
  Mean RMSE: 90.1887 (Std Dev: 29.9654)

Metrics for Target: average_voltage
  Mean R-squared (R²): 0.3901 (Std Dev: 0.3152)
  Mean MAE: 0.5661 (Std Dev: 0.2717)
  Mean RMSE: 4.8432 (Std Dev: 4.9925)

Metrics for Target: energy_above_hull
  Mean R-squared (R²): 0.4469 (Std Dev: 0.0193)
  Mean MAE: 0.0247 (Std Dev: 0.0005)
  Mean RMSE: 0.0337 (Std Dev: 0.0008)


## Testing LightGBM model on test set

In [ ]:
# Train on the full training set and evaluate on test set for a final view
print("\n LightGBM Model Evaluation on Test Set")
multioutput_lgb_final = MultiOutputRegressor(lgb_base_model)
multioutput_lgb_final.fit(X_train_scaled, Y_train) # Train on full training data

Y_pred_lgb_test = multioutput_lgb_final.predict(X_test_scaled)
Y_pred_lgb_test_df = pd.DataFrame(Y_pred_lgb_test, columns=Y_test.columns, index=Y_test.index)

for i, target_col in enumerate(Y_test.columns):
    r2 = r2_score(Y_test[target_col], Y_pred_lgb_test_df[target_col])
    mae = mean_absolute_error(Y_test[target_col], Y_pred_lgb_test_df[target_col])
    rmse = np.sqrt(mean_squared_error(Y_test[target_col], Y_pred_lgb_test_df[target_col]))

    print(f"\nMetrics for Target: {target_col} (Test Set)")
    print(f"  R-squared (R²): {r2:.4f}")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")




 LightGBM Model Evaluation on Test Set
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001228 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4497
[LightGBM] [Info] Number of data points in the train set: 2756, number of used features: 49
[LightGBM] [Info] Start training from score 111.441483
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001214 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4497
[LightGBM] [Info] Number of data points in the train set: 2756, number of used features: 49
[LightGBM] [Info] Start training from score 2.504560
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001206 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4497
[LightGBM] [Info] Number of data points in the train set: 2756, number of used f

## Hyperparameter tuning for LightGBM model

In [ ]:
# Define the base LightGBM model for tuning
lgb_base_model_tuned = lgb.LGBMRegressor(
    objective='regression_l1', # MAE objective
    metric='mae',              # Metric for evaluation during training
    random_state=9,
    n_jobs=-1,
    boosting_type='gbdt'
)

# Wrap it in MultiOutputRegressor
multioutput_lgb_tuned = MultiOutputRegressor(lgb_base_model_tuned)

# Define the parameter distribution for RandomizedSearchCV for LightGBM
param_distributions_lgb = {
    'estimator__n_estimators': [100, 200, 300], # Number of boosting rounds
    'estimator__learning_rate': [0.01, 0.05, 0.1], # Step size shrinkage
    'estimator__num_leaves': [10, 20, 40], # Maximum number of leaves
    'estimator__max_depth': [-1, 5, 8, 10, 12], # Maximum depth or -1 for no limit
    'estimator__min_child_samples': [10, 20, 30], # Minimum data needed in a child
    'estimator__subsample': [0.6, 0.7, 0.8, 0.9, 1.0], # Subsample ratio of the training instance
    'estimator__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0], # Subsample ratio of columns
    'estimator__reg_alpha': [0, 0.005, 0.01, 0.05, 0.1, 0.5, 1], # L1 regularization
    'estimator__reg_lambda': [0, 0.005, 0.01, 0.05, 0.1, 0.5, 1] # L2 regularization
}

# Initialize RandomizedSearchCV for LightGBM
random_search_lgb = RandomizedSearchCV(
    estimator=multioutput_lgb_tuned,
    param_distributions=param_distributions_lgb,
    n_iter=25, # Number of parameter settings that are sampled
    cv=4,      # Number of folds in cross-validation
    verbose=2,
    random_state=9,
    n_jobs=-1  # Use all available CPU cores
)

# Fit RandomizedSearchCV to the scaled training data and targets
print("\n--- Performing Randomized Search for LightGBM Hyperparameter Tuning ---")
random_search_lgb.fit(X_train_scaled, Y_train)

# Get the best parameters and the best score
best_params_lgb = random_search_lgb.best_params_
best_score_lgb = random_search_lgb.best_score_ # This is the mean score of the best estimator on the held out data.

print(f"\nBest parameters found for LightGBM: {best_params_lgb}")
print(f"Best score found for LightGBM (mean CV score): {best_score_lgb:.4f}")




--- Performing Randomized Search for LightGBM Hyperparameter Tuning ---
Fitting 4 folds for each of 25 candidates, totalling 100 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4497
[LightGBM] [Info] Number of data points in the train set: 2756, number of used features: 49
[LightGBM] [Info] Start training from score 111.441483
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001267 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4497
[LightGBM] [Info] Number of data points in the train set: 2756, number of used features: 49
[LightGBM] [Info] Start training from score 2.504560
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001239 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] T

In [ ]:
# Re-evaluate the best model with cross-validation on all targets for robust metrics

# Get the best estimator from the search
best_multioutput_lgb = random_search_lgb.best_estimator_

# Prepare lists to store scores for each target across all folds
lgb_tuned_target_r2_scores = {col: [] for col in Y_train.columns}
lgb_tuned_target_mae_scores = {col: [] for col in Y_train.columns}
lgb_tuned_target_rmse_scores = {col: [] for col in Y_train.columns}

# Perform cross-validation with the best estimator
kfold = KFold(n_splits=5, shuffle=True, random_state=9)

print("\n--- Cross-Validation Results for Tuned LightGBM ---")
for fold_idx, (train_index, test_index) in enumerate(kfold.split(X_train_scaled, Y_train)):
    X_train_fold, X_test_fold = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]

    print(f"Fold {fold_idx + 1}/5: Training tuned model...")
    # Fit a new instance of the best model for each fold (important for proper CV)
    fold_lgb_base_model_tuned = lgb.LGBMRegressor(
        objective='regression_l1', # MAE objective
        metric='mae',              # Metric for evaluation during training
        n_estimators=best_params_lgb['estimator__n_estimators'],
        learning_rate=best_params_lgb['estimator__learning_rate'],
        num_leaves=best_params_lgb['estimator__num_leaves'],
        max_depth=best_params_lgb['estimator__max_depth'],
        min_child_samples=best_params_lgb['estimator__min_child_samples'],
        subsample=best_params_lgb['estimator__subsample'],
        colsample_bytree=best_params_lgb['estimator__colsample_bytree'],
        reg_alpha=best_params_lgb['estimator__reg_alpha'],
        reg_lambda=best_params_lgb['estimator__reg_lambda'],
        random_state=9,
        n_jobs=-1,
        boosting_type='gbdt'
    )
    fold_multioutput_lgb_tuned = MultiOutputRegressor(fold_lgb_base_model_tuned)
    fold_multioutput_lgb_tuned.fit(X_train_fold, Y_train_fold)

    Y_pred_fold = fold_multioutput_lgb_tuned.predict(X_test_fold)
    Y_pred_fold_df = pd.DataFrame(Y_pred_fold, columns=Y_test_fold.columns, index=Y_test_fold.index)

    print(f"Fold {fold_idx + 1}/5: Evaluating tuned model...")
    for i, target_col in enumerate(Y_test_fold.columns):
        r2 = r2_score(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        mae = mean_absolute_error(Y_test_fold[target_col], Y_pred_fold_df[target_col])
        rmse = np.sqrt(mean_squared_error(Y_test_fold[target_col], Y_pred_fold_df[target_col]))

        lgb_tuned_target_r2_scores[target_col].append(r2)
        lgb_tuned_target_mae_scores[target_col].append(mae)
        lgb_tuned_target_rmse_scores[target_col].append(rmse)


--- Cross-Validation Results for Tuned LightGBM ---
Fold 1/5: Training tuned model...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001075 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4415
[LightGBM] [Info] Number of data points in the train set: 2204, number of used features: 49
[LightGBM] [Info] Start training from score 111.869675
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4415
[LightGBM] [Info] Number of data points in the train set: 2204, number of used features: 49
[LightGBM] [Info] Start training from score 2.491324
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000977 seconds.
You can set `force_col_wise=true` to remove the ov

In [ ]:
# Print aggregated cross-validation results for the tuned LightGBM
print("\n--- Tuned LightGBM Cross-Validation Results (Mean across 5 folds, Per Target) ---")
for target_col in Y_train.columns:
    print(f"\nMetrics for Target: {target_col}")
    print(f"  Mean R-squared (R²): {np.mean(lgb_tuned_target_r2_scores[target_col]):.4f} (Std Dev: {np.std(lgb_tuned_target_r2_scores[target_col]):.4f})")
    print(f"  Mean MAE: {np.mean(lgb_tuned_target_mae_scores[target_col]):.4f} (Std Dev: {np.std(lgb_tuned_target_mae_scores[target_col]):.4f})")
    print(f"  Mean RMSE: {np.mean(lgb_tuned_target_rmse_scores[target_col]):.4f} (Std Dev: {np.std(lgb_tuned_target_rmse_scores[target_col]):.4f})")




--- Tuned LightGBM Cross-Validation Results (Mean across 5 folds, Per Target) ---

Metrics for Target: theoretical_capacity_grav
  Mean R-squared (R²): 0.7881 (Std Dev: 0.0630)
  Mean MAE: 23.6727 (Std Dev: 2.4109)
  Mean RMSE: 65.4221 (Std Dev: 12.6052)

Metrics for Target: average_voltage
  Mean R-squared (R²): 0.5229 (Std Dev: 0.2852)
  Mean MAE: 0.5522 (Std Dev: 0.2745)
  Mean RMSE: 4.4684 (Std Dev: 4.9971)

Metrics for Target: energy_above_hull
  Mean R-squared (R²): 0.4270 (Std Dev: 0.0202)
  Mean MAE: 0.0252 (Std Dev: 0.0005)
  Mean RMSE: 0.0343 (Std Dev: 0.0009)


## Testing tuned LightGBM model on test set

In [ ]:
# Final Evaluation on the untouched Test Set with the best tuned model
print("\n--- Final Evaluation of Tuned LightGBM Model on Test Set ---")
Y_pred_tuned_lgb_test = best_multioutput_lgb.predict(X_test_scaled)
Y_pred_tuned_lgb_test_df = pd.DataFrame(Y_pred_tuned_lgb_test, columns=Y_test.columns, index=Y_test.index)

for i, target_col in enumerate(Y_test.columns):
    r2 = r2_score(Y_test[target_col], Y_pred_tuned_lgb_test_df[target_col])
    mae = mean_absolute_error(Y_test[target_col], Y_pred_tuned_lgb_test_df[target_col])
    rmse = np.sqrt(mean_squared_error(Y_test[target_col], Y_pred_tuned_lgb_test_df[target_col]))

    print(f"\nMetrics for Target: {target_col} (Test Set, Tuned LightGBM)")
    print(f"  R-squared (R²): {r2:.4f}")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")




--- Final Evaluation of Tuned LightGBM Model on Test Set ---

Metrics for Target: theoretical_capacity_grav (Test Set, Tuned LightGBM)
  R-squared (R²): 0.8093
  Mean Absolute Error (MAE): 18.0919
  Root Mean Squared Error (RMSE): 31.1952

Metrics for Target: average_voltage (Test Set, Tuned LightGBM)
  R-squared (R²): 0.5223
  Mean Absolute Error (MAE): 0.4794
  Root Mean Squared Error (RMSE): 2.7391

Metrics for Target: energy_above_hull (Test Set, Tuned LightGBM)
  R-squared (R²): 0.4505
  Mean Absolute Error (MAE): 0.0247
  Root Mean Squared Error (RMSE): 0.0342


## Save Tuned LightGBM model

In [ ]:
# Define the path for the models folder
model_folder_path = '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model'
os.makedirs(model_folder_path, exist_ok=True) # Create the directory if it doesn't exist

# Define the filename for the tuned LightGBM model
lgb_model_filename_tuned = os.path.join(model_folder_path, "tuned_lightgbm_multi_output_model_v2.joblib")

try:
    # Save the best tuned LightGBM model
    joblib.dump(best_multioutput_lgb, lgb_model_filename_tuned)

    print(f"Tuned LightGBM model saved to: '{lgb_model_filename_tuned}'")

except Exception as e:
    print(f"Error saving tuned LightGBM model: {e}")


Tuned LightGBM model saved to: '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/tuned_lightgbm_multi_output_model_v2.joblib'


# Appendix

This appendix provides additional details and resources related to the analysis performed in this notebook.

## A. Models

Details about the models trained and evaluated in this notebook.

### A.1 XGBoost Model

- **Base Model Configuration:**
  - Objective: `reg:squarederror`
  - n_estimators: 300
  - learning_rate: 0.1
  - max_depth: 6
  - subsample: 0.8
  - colsample_bytree: 0.8
  - random_state: 9
  - n_jobs: -1

- **Tuned Model Hyperparameters:**
  - estimator__subsample: 0.8
  - estimator__reg_lambda: 1
  - estimator__reg_alpha: 0
  - estimator__n_estimators: 200
  - estimator__max_depth: 8
  - estimator__learning_rate: 0.15
  - estimator__gamma: 0
  - estimator__colsample_bytree: 1.0

- **Cross-Validation Results (Tuned Model):**
  - theoretical_capacity_grav:
    - Mean R²: 0.8705 (Std Dev: 0.0248)
    - Mean MAE: 19.9334 (Std Dev: 1.4438)
    - Mean RMSE: 52.2823 (Std Dev: 12.1327)
  - average_voltage:
    - Mean R²: 0.8067 (Std Dev: 0.1540)
    - Mean MAE: 0.4362 (Std Dev: 0.1741)
    - Mean RMSE: 2.8908 (Std Dev: 3.6284)
  - energy_above_hull:
    - Mean R²: 0.4592 (Std Dev: 0.0350)
    - Mean MAE: 0.0246 (Std Dev: 0.0009)
    - Mean RMSE: 0.0333 (Std Dev: 0.0011)

- **Test Set Evaluation (Tuned Model):**
  - theoretical_capacity_grav:
    - R²: 0.6534
    - MAE: 18.2208
    - RMSE: 42.0546
  - average_voltage:
    - R²: 0.8449
    - MAE: 0.3676
    - RMSE: 1.5609
  - energy_above_hull:
    - R²: 0.5002
    - MAE: 0.0236
    - RMSE: 0.0326

### A.2 Random Forest Model

- **Base Model Configuration:**
  - n_estimators: 300
  - max_depth: 10
  - min_samples_split: 5
  - min_samples_leaf: 3
  - random_state: 9
  - n_jobs: -1

- **Tuned Model Hyperparameters:**
  - estimator__n_estimators': 200
  - estimator__min_samples_split': 2
  - estimator__min_samples_leaf': 1
  - estimator__max_depth': 15
  - estimator__bootstrap': True

- **Cross-Validation Results (Tuned Model):**
  - theoretical_capacity_grav:
    - Mean R²: 0.8114 (Std Dev: 0.0472)
    - Mean MAE: 22.7258 (Std Dev: 1.6363)
    - Mean RMSE: 62.5175 (Std Dev: 15.0600)
  - average_voltage:
    - Mean R²: 0.7934 (Std Dev: 0.1878)
    - Mean MAE: 0.4743 (Std Dev: 0.1984)
    - Mean RMSE: 3.0983 (Std Dev: 4.0410)
  - energy_above_hull:
    - Mean R²: 0.4366 (Std Dev: 0.0128)
    - Mean MAE: 0.0256 (Std Dev: 0.0004)
    - Mean RMSE: 0.0340 (Std Dev: 0.0005)

- **Test Set Evaluation (Tuned Model):**
  - theoretical_capacity_grav:
    - R²: 0.6406
    - MAE: 19.0836
    - RMSE: 42.8224
  - average_voltage:
    - R²: 0.8924
    - MAE: 0.3814
    - RMSE: 1.3002
  - energy_above_hull:
    - R²: 0.4496
    - MAE: 0.0251
    - RMSE: 0.0342

### A.3 LightGBM Model

- **Base Model Configuration:**
  - objective: 'regression_l1'
  - metric: 'mae'
  - n_estimators: 300
  - learning_rate: 0.1
  - num_leaves: 31
  - max_depth: -1
  - min_child_samples: 20
  - subsample: 0.8
  - colsample_bytree: 0.8
  - random_state: 9
  - n_jobs: -1
  - boosting_type: 'gbdt'

- **Tuned Model Hyperparameters:**
  - estimator__subsample: 0.7
  - estimator__reg_lambda: 0
  - estimator__reg_alpha: 0.01
  - estimator__num_leaves: 20
  - estimator__n_estimators: 300
  - estimator__min_child_samples: 10
  - estimator__max_depth: 10
  - estimator__learning_rate: 0.1
  - estimator__colsample_bytree: 0.7

- **Cross-Validation Results (Tuned Model):**
  - theoretical_capacity_grav:
    - Mean R²: 0.7881 (Std Dev: 0.0630)
    - Mean MAE: 23.6727 (Std Dev: 2.4109)
    - Mean RMSE: 65.4221 (Std Dev: 12.6052)
  - average_voltage:
    - Mean R²: 0.5229 (Std Dev: 0.2852)
    - Mean MAE: 0.5522 (Std Dev: 0.2745)
    - Mean RMSE: 4.4684 (Std Dev: 4.9971)
  - energy_above_hull:
    - Mean R²: 0.4270 (Std Dev: 0.0202)
    - Mean MAE: 0.0252 (Std Dev: 0.0005)
    - Mean RMSE: 0.0343 (Std Dev: 0.0009)

- **Test Set Evaluation (Tuned Model):**
  - theoretical_capacity_grav:
    - R²: 0.8093
    - MAE: 18.0919
    - RMSE: 31.1952
  - average_voltage:
    - R²: 0.5223
    - MAE: 0.4794
    - RMSE: 2.7391
  - energy_above_hull:
    - R²: 0.4505
    - MAE: 0.0247
    - RMSE: 0.0342

## B. Data

Information about the dataset used and processed in this notebook.

- **Original Data Source:** `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/battery_materials_engineered_features_v2.csv`
- **Shape of Engineered DataFrame:** (3445, 53)
- **Defined Features (X):** 45 columns (list of columns provided in notebook)
- **Defined Targets (Y):** 3 columns (`theoretical_capacity_grav`, `average_voltage`, `energy_above_hull`)
- **Train-Test Split:** 80% Train (2756 samples), 20% Test (689 samples)
- **Scaled Features:** 31 numerical columns were scaled using `StandardScaler`.
- **Added Interaction Features:** 4 new features (`exp_band_gap`, `exp_energy_vol`, `theo_band_gap`, `theo_energy_vol`) added to scaled features.
- **Final Scaled Features Shape:** (3445, 49)
- **Saved Training/Test Data:**
    - X_train_scaled.csv
    - Y_train.csv
    - X_test_scaled.csv
    - Y_test.csv
    - Saved to: `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/Training_Data`

## C. Code

Relevant code snippets and libraries used.

- **Libraries Imported:** `pandas`, `numpy`, `os`, `sys`, `lightgbm`, `sklearn.model_selection`, `sklearn.preprocessing`, `sklearn.metrics`, `joblib`, `sklearn.multioutput`, `sklearn.ensemble`, `xgboost`, `random`
- **Key Functions/Techniques:**
    - `train_test_split`: For splitting data into training and testing sets.
    - `StandardScaler`: For standardizing numerical features.
    - `MultiOutputRegressor`: For handling multi-target regression with base estimators (XGBoost, RandomForest, LightGBM).
    - `RandomizedSearchCV`: For hyperparameter tuning.
    - `r2_score`, `mean_absolute_error`, `mean_squared_error`: For evaluating model performance.
    - `joblib.dump`: For saving trained models and the scaler.
    - Addition of interaction features through multiplication of existing columns.

## D. Saved Files

List of files saved during the execution of this notebook.

- **Model Files:**
    - `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/tuned_xgboost_multi_output_model_v2.joblib`
    - `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/scaler_X_v2.joblib`
    - `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/feature_column_names_v2.joblib`
    - `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/tuned_random_forest_multi_output_model_v2.joblib`
    - `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Model/tuned_lightgbm_multi_output_model_v2.joblib`

- **Training/Test Data Files:**
    - `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/Training_Data/X_train_scaled.csv`
    - `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/Training_Data/Y_train.csv`
    - `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/Training_Data/X_test_scaled.csv`
    - `/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/Training_Data/Y_test.csv`